In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
os.listdir(path)

['test',
 'new plant diseases dataset(augmented)',
 'New Plant Diseases Dataset(Augmented)']

In [ ]:

train_path = os.path.join(path,"new plant diseases dataset(augmented)","New Plant Diseases Dataset(Augmented)",'train')
valid_path = os.path.join(path,"new plant diseases dataset(augmented)","New Plant Diseases Dataset(Augmented)",'valid')
test_path = os.path.join(path,'test',"test")


In [ ]:
names = os.listdir(train_path)

values = [i for i in range(len(names))]
classes_index = {value:key for (key,value) in zip(names,values)}
classes_names= {key:value for (key,value) in zip(names,values)}

## Data Preparation

In [ ]:
# Data augmentation & preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Load data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),  # VGG input size
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


## Model Building (Using transfer Learning)

In [ ]:
# Load pre-trained VGG16 (without top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional layers (optional)
for layer in base_model.layers:
    layer.trainable = False  # Freeze to avoid overfitting

In [ ]:
# Add custom classification head
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer
])



In [ ]:
# Compile
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Train
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32
)

# Save model
model.save('my_vgg_model.h5')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1086s 488ms/step - accuracy: 0.3535 - loss: 2.3670 - val_accuracy: 0.8543 - val_loss: 0.5931
Epoch 2/10
   1/2196 ━━━━━━━━━━━━━━━━━━━━ 6:05 166ms/step - accuracy: 0.7812 - loss: 0.8308

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 96s 44ms/step - accuracy: 0.7812 - loss: 0.8308 - val_accuracy: 0.8547 - val_loss: 0.5907
Epoch 3/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1103s 502ms/step - accuracy: 0.6666 - loss: 1.1047 - val_accuracy: 0.8854 - val_loss: 0.4136
Epoch 4/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 142s 65ms/step - accuracy: 0.8125 - loss: 0.6109 - val_accuracy: 0.8861 - val_loss: 0.4129
Epoch 5/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1102s 485ms/step - accuracy: 0.7246 - loss: 0.8863 - val_accuracy: 0.8940 - val_loss: 0.3554
Epoch 6/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 96s 44ms/step - accuracy: 0.7812 - loss: 0.8392 - val_accuracy: 0.8929 - val_loss: 0.3576
Epoch 7/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1030s 468ms/step - accuracy: 0.7604 - loss: 0.7615 - val_accuracy: 0.9043 - val_loss: 0.3128
Epoch 8/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 96s 44ms/step - accuracy: 0.8438 - loss: 0.4734 - val_accuracy: 0.9044 - val_loss: 0.3123
Epoch 9/10
2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1043s 475ms/step - accuracy: 0.7814

In [ ]:
test_imgs = []
root = os.path.dirname(test_path)
for file in os.listdir(test_path):
    file_source = os.path.join(root,file)
    test_imgs.append(file_source)
test_imgs[0]

In [ ]:

def predict_image(img_path):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize (same as training)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)

    return predicted_class, confidence

# Example usage
img_path = train_imgs[0]
predicted_class, confidence = predict_image(img_path)
print(f"Predicted: {predicted_class} (Confidence: {confidence:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted: 22 (Confidence: 1.00)


In [ ]:
classes_names

{'Potato___healthy': 0,
 'Raspberry___healthy': 1,
 'Soybean___healthy': 2,
 'Potato___Late_blight': 3,
 'Strawberry___Leaf_scorch': 4,
 'Apple___Cedar_apple_rust': 5,
 'Potato___Early_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Cherry_(including_sour)___Powdery_mildew': 8,
 'Peach___Bacterial_spot': 9,
 'Tomato___Tomato_mosaic_virus': 10,
 'Cherry_(including_sour)___healthy': 11,
 'Peach___healthy': 12,
 'Tomato___Spider_mites Two-spotted_spider_mite': 13,
 'Apple___Black_rot': 14,
 'Corn_(maize)___Common_rust_': 15,
 'Apple___Apple_scab': 16,
 'Corn_(maize)___healthy': 17,
 'Squash___Powdery_mildew': 18,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 19,
 'Corn_(maize)___Northern_Leaf_Blight': 20,
 'Tomato___Septoria_leaf_spot': 21,
 'Grape___healthy': 22,
 'Pepper,_bell___Bacterial_spot': 23,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 24,
 'Grape___Black_rot': 25,
 'Blueberry___healthy': 26,
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 27,
 'Tomato___Bacterial_spot': 28,
 'Tom

In [ ]:
train_imgs[0]

'/root/.cache/kagglehub/datasets/vipoooool/new-plant-diseases-dataset/versions/2/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/Potato___healthy/20ac28d6-5708-4e31-8676-1b1979de094f___RS_HL 1735_new30degFlipTB.JPG'

In [ ]:
classes_index

{0: 'Potato___healthy',
 1: 'Raspberry___healthy',
 2: 'Soybean___healthy',
 3: 'Potato___Late_blight',
 4: 'Strawberry___Leaf_scorch',
 5: 'Apple___Cedar_apple_rust',
 6: 'Potato___Early_blight',
 7: 'Tomato___Leaf_Mold',
 8: 'Cherry_(including_sour)___Powdery_mildew',
 9: 'Peach___Bacterial_spot',
 10: 'Tomato___Tomato_mosaic_virus',
 11: 'Cherry_(including_sour)___healthy',
 12: 'Peach___healthy',
 13: 'Tomato___Spider_mites Two-spotted_spider_mite',
 14: 'Apple___Black_rot',
 15: 'Corn_(maize)___Common_rust_',
 16: 'Apple___Apple_scab',
 17: 'Corn_(maize)___healthy',
 18: 'Squash___Powdery_mildew',
 19: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 20: 'Corn_(maize)___Northern_Leaf_Blight',
 21: 'Tomato___Septoria_leaf_spot',
 22: 'Grape___healthy',
 23: 'Pepper,_bell___Bacterial_spot',
 24: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 25: 'Grape___Black_rot',
 26: 'Blueberry___healthy',
 27: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 28: 'Tomato___Bacterial_spot',
 29: 